In [1]:
!pip install sentence_transformers
!pip install torch_geometric
!pip install opentsne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Graph_ML_Proj

/content/drive/MyDrive/Graph_ML_Proj


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
from functions import *

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import random
import numpy as np
from matplotlib import pyplot

import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv

# adjust this import accordingly to how you call the script
# from functions import *
from torch.utils.data import DataLoader

# evaluation
from sklearn.metrics import precision_score, recall_score, f1_score

# visualise learnt intermediate embeddings
import matplotlib.pyplot as plt
from openTSNE import TSNE
# from sklearn.manifold import TSNE # very slow
tsne = TSNE(
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# visualise learnt output embeddings
def visualize(h, color):
    # z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())
    z = tsne.fit(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    print(f"Number of positive is {sum(color)}")
    print(f"Total number is {len(color)}")
    s = [0.5 if c==0 else 7 for c in color]

    plt.scatter(z[:, 0], z[:, 1], s=1, c=color, cmap=pyplot.jet())
    plt.show()

In [ ]:
# train code
def train(dataloader):
    """
    one epoch
    returns average loss for one epoch
    """
    model.train()
    total_loss = 0

    # loop batches from dataloader
    for d, (batched_subgraphs, labels) in enumerate(dataloader):

        optimizer.zero_grad()

        batched_subgraphs = batched_subgraphs.to(device)
        labels = [label.to(device) for label in labels]

        # forward pass
        out, x_inter = model(batched_subgraphs)

        # calculate loss
        batch_loss = 0
        for i, label in enumerate(labels):
            node_mask = (batched_subgraphs.batch == i)
            logits = out[node_mask]
            target = label.float()

            pos_weight = torch.tensor([len(target) / target.sum()], device=device)
            loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
            batch_loss += loss_fn(logits, target)

        # backward pass and optimization step
        batch_loss.backward()
        optimizer.step()

        # shift back to cpu
        batch_loss = batch_loss.detach().cpu()
        output_cpu = out.detach().cpu()
        x_inter_cpu = x_inter.detach().cpu()
        labels_cpu = [label.detach().cpu() for label in labels]

        total_loss += batch_loss.item()

        # print batch loss every 5 steps
        if d%5 == 0:
            print('Batch loss is', batch_loss.item())

        # # visualise last batch in the epoch
        # if d == len(dataloader) - 1:
        #     visualize(x_inter_cpu, color=torch.cat(labels_cpu, dim=0))

    torch.cuda.empty_cache() # help clear cache taking up cuda space

    return total_loss / len(dataloader)

# evaluation code
def evaluate(dataloader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batched_subgraphs, labels in dataloader:
            batched_subgraphs = batched_subgraphs.to(device)
            labels = [label for label in labels]

            out, _ = model(batched_subgraphs)
            output_cpu = out.detach().cpu()

            # calculate accuracy for each subgraph
            for i, label in enumerate(labels):
                node_mask = (batched_subgraphs.batch == i).detach().cpu()
                preds = (torch.sigmoid(output_cpu[node_mask]) > 0.5).int()

                all_preds.extend(preds.tolist())
                all_labels.extend(label.tolist())
                correct += (preds == label).sum().item()
                total += label.size(0)

    precision = precision_score(all_labels, all_preds, average='binary') # for positive class
    recall = recall_score(all_labels, all_preds, average='binary') # for positive class
    f1 = f1_score(all_labels, all_preds, average='binary') # for positive class
    accuracy = correct / total # biased towards 0

    torch.cuda.empty_cache() # help clear cache taking up cuda space

    return accuracy, precision, recall, f1, all_preds, all_labels

In [ ]:
# not sure
def focal_loss(logits, labels, alpha=0.25, gamma=2.0):
    """
    NOT USED.
    logits: Predicted output from the model (after log-softmax).
    labels: Ground truth labels (0 or 1).
    alpha: Balancing factor for the minority class.
    gamma: Focusing parameter for adjusting the rate at which easy examples are down-weighted.
    """
    # Compute cross-entropy loss per example
    ce_loss = F.nll_loss(logits, labels, reduction='none')
    # Probabilities for each example
    pt = torch.exp(-ce_loss)
    # Apply the focal loss adjustment
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss

    # Return the mean loss
    return focal_loss.mean()

In [ ]:
# load data
path_to_node_embed = 'Datasets/MetaQA_dataset/processed/node2vec_embeddings/ud_node2vec_embeddings.txt'
path_to_idxes = 'Datasets/MetaQA_dataset/processed/idxes.json'
path_to_qa = 'Datasets/MetaQA_dataset/vanilla 3-hop/qa_train.txt'

# train
data = KGQADataset(path_to_node_embed, path_to_idxes, path_to_qa)
sub_data1 = torch.utils.data.Subset(data, list(range(3000)))
dataloader_train = DataLoader(sub_data1, batch_size=32, collate_fn=collate_fn, shuffle=True)
# some from train to evaluate
sub_data2 = torch.utils.data.Subset(data, list(range(5000, 5000+400)))
dataloader_val = DataLoader(sub_data2, batch_size=32, collate_fn=collate_fn, shuffle=True)

# some from test to evaluate
test = KGQADataset(path_to_node_embed, path_to_idxes, 'Datasets/MetaQA_dataset/vanilla 3-hop/qa_test.txt')
sub_data3 = torch.utils.data.Subset(test, list(range(400)))
dataloader_test = DataLoader(sub_data3, batch_size=32, collate_fn=collate_fn, shuffle=True)

### Test training RGCN

-   Replace GCNConv layers with RGCNConv layers. Each RGCNConv layer can accept argument num_relations, specifying the number of unique edge types.
-   Adjust forward method to include edge_attr (of type `Long`), where each unique relation is mapped to an integer.

In [ ]:
torch.manual_seed(2024)
random.seed(2024)
np.random.seed(2024)

In [ ]:
# model
class RGCN(torch.nn.Module):
    def __init__(self, node_dim, question_dim, hidden_dim, num_relations, output_dim=1):
        super(RGCN, self).__init__()
        # 4 layers, 64 hidden_dim
        self.conv1 = RGCNConv(node_dim + question_dim, hidden_dim, num_relations)
        self.conv2 = RGCNConv(hidden_dim, hidden_dim, num_relations)
        self.conv3 = RGCNConv(hidden_dim, hidden_dim, num_relations)
        self.conv4 = RGCNConv(hidden_dim, output_dim, num_relations)

    def forward(self, batched_subgraphs):
        # concatenate question embeddings with node features for each subgraph along feature dimension
        question_emb_expanded = []
        for subgraph in batched_subgraphs.to_data_list():
            subgraph.x = torch.cat((subgraph.x, subgraph.qn.unsqueeze(0).expand(subgraph.x.size(0), -1)), dim=1)
            question_emb_expanded.append(subgraph.x)

        batched_subgraphs.x = torch.cat(question_emb_expanded, dim=0)
        x, edge_index = batched_subgraphs.x, batched_subgraphs.edge_index
        edge_attr = batched_subgraphs.edge_attr  # Get edge attributes

        # RGCN
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x_inter = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x_inter)
        x = self.conv4(x, edge_index, edge_attr)
        # Output logits directly for BCEWithLogitsLoss
        return x.squeeze(-1), x_inter

# Binary classification (answer candidate or not)
model = RGCN(node_dim=64, question_dim=384, hidden_dim=64, num_relations=9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
# train 2 epoch
for epoch in range(2):
    loss = train(dataloader_train)
    val_accuracy, val_p, val_r, val_f1, _, _ = evaluate(dataloader_val)
    test_accuracy, test_p, test_r, test_f1, _, _ = evaluate(dataloader_test)
    print(f'Epoch {epoch}, Train Loss: {loss}')
    print(f'Validation Accuracy: {val_accuracy:.8f}, Validation P/R/F1: {val_p:.3f}/{val_r:.3f}/{val_f1:.3f}')
    print(f'Test Accuracy: {test_accuracy:.8f}, Test P/R/F1: {test_p:.3f}/{test_r:.3f}/{test_f1:.3f}')

Batch loss is 53.67695617675781
Batch loss is 26.509693145751953
Batch loss is 22.01544761657715
Batch loss is 24.990270614624023
Batch loss is 24.2110595703125
Batch loss is 15.868939399719238
Batch loss is 16.62416648864746
Batch loss is 15.463056564331055
Batch loss is 14.899299621582031
Batch loss is 17.958518981933594
Batch loss is 17.1336727142334
Batch loss is 15.919134140014648
Batch loss is 15.834423065185547
Batch loss is 14.607815742492676
Batch loss is 12.209258079528809
Batch loss is 16.970762252807617
Batch loss is 15.018385887145996
Batch loss is 11.298216819763184
Batch loss is 14.600522994995117
Epoch 0, Train Loss: 18.280563740020103
Validation Accuracy: 0.86071343, Validation P/R/F1: 0.008/0.940/0.016
Test Accuracy: 0.84633650, Test P/R/F1: 0.007/0.933/0.014
Batch loss is 15.942140579223633
Batch loss is 15.089784622192383
Batch loss is 11.373772621154785
Batch loss is 10.183561325073242
Batch loss is 13.38595199584961
Batch loss is 14.635862350463867
Batch loss is 1